In [2]:
"""
Initial software, Jean-Daniel Fekete, Christian Poli, Copyright (c) Inria, BSD 3-Clause License, 2021, v0.1.0

This notebook contains substantial portions of https://github.com/vega/ipyvega/blob/a83a96bc5209f59b646ef12d5a1fbb5fb971b45a/ui-tests/tests/notebooks/vega.ipynb , which has the following license:

Copyright 2015 Interactive Data Lab and contributors

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS 
FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, 
BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, 
STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
"""
spec_no_data = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {"name": "data"},
  "mark": "bar",
  "encoding": {
    "x": {"aggregate": "sum", "field": "yield"},
    "y": {"field": "variety"},
    "color": {"field": "site"}
  }
}
from ipyprogressivis.widgets import VegaWidget
import requests
import json
req = requests.get("https://raw.githubusercontent.com/vega/vega/master/docs/data/barley.json")
values = json.loads(req.text)
#data
widget = VegaWidget(spec=spec_no_data)
display(widget)
widget.update('data', insert=values)

VegaWidget()

In [3]:
import pandas as pd
df = pd.read_json("https://raw.githubusercontent.com/vega/vega/master/docs/data/barley.json")
widget = VegaWidget(spec=spec_no_data)
display(widget)
widget.update("data", insert=df)

VegaWidget()

In [4]:
widget = VegaWidget(spec=spec_no_data)
widget.compression = 'zlib'
display(widget)
widget.update('data', insert=df)

VegaWidget()

In [5]:
from ipytablewidgets import LZ4Compressor
widget = VegaWidget(spec=spec_no_data)
widget.compression = LZ4Compressor(2)
display(widget)
widget.update('data', insert=df)

VegaWidget()

In [6]:
import numpy as np
import asyncio as aio
nbins = 50
spec_no_data = {
    '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
     'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
     'data': {'name': 'data'},
     'encoding': {
                  'x': {'field': 'x', 'type': 'ordinal'},
                  'y': {'field': 'y', 'type': 'quantitative'}
             },
     'mark': 'bar'}
spec_with_data = spec_no_data.copy()
nsamples = 100_000
steps = 1
np.random.seed(42)
rdata = np.random.normal(loc=0, scale=0.1, size=nsamples)
widget = VegaWidget(spec=spec_no_data)
display(widget)
async def progressive_loop(widget, nbins=50):
    hist = np.zeros(nbins)
    for sl in np.split(rdata, steps):
        await aio.sleep(0.1)
        h, _ = np.histogram(sl, bins=nbins, range=(-1.0,1.0))
        hist += h
        source = pd.DataFrame({
            'x': range(nbins),
            'y': hist,
        }, dtype='float32')
        widget._displayed = True
        widget.update('data', insert=source, remove="true")

await progressive_loop(widget)

VegaWidget()

In [7]:
from ipytablewidgets import LZ4Compressor
import time
nbins = 256
spec_no_data = {'$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 "width": 500,
 "height": 400,
 'data': {'name': 'data'},
 'encoding': {'color': {'field': 'z', 
                        'type': 'quantitative',
                        # "scale": {
                        # "domain": [0,1]  # Remove if domain changes
                        # }
                        },
  'x': {'field': 'x', 'type': 'ordinal'},
  'y': {'field': 'y', 'type': 'ordinal'}},
 'mark': 'rect',
 "config": {
    "axis": {
      "disable": True  # Change to False to see the ticks
    }
  }}
widget = VegaWidget(spec=spec_no_data)
widget.compression = LZ4Compressor(2)
steps = 1
display(widget)
nsamples=100_000
means = [0.1, 0.3]
cov = [[0.01, 0], [0, 0.09]]
rdata = np.random.multivariate_normal(means, cov, size=(nsamples))
async def progressive_loop(widget, nbins=nbins):
    hist = np.zeros((nbins,nbins), dtype='uint32')
    times = []
    for sl in np.split(rdata, steps):
        await aio.sleep(0)
        h, *_ = np.histogram2d(sl[:,0],sl[:,1], bins=nbins, range=((-1.0,1.0),(-1.0,1.0)))
        hist += h.astype('uint32')
        widget._displayed = True
        value = hist
        widget._displayed = True
        start = time.time()
        widget.update('data', insert=value, remove="true")  

await progressive_loop(widget, nbins)

VegaWidget()